In [1]:
import jax
import jax.numpy as jnp

In [2]:
import jax
import jax.numpy as jnp

global_list = []

def log2(x):
  global_list.append(x)
  ln_x = jnp.log(x)
  ln_2 = jnp.log(2.0)
  return ln_x / ln_2

print(jax.make_jaxpr(log2)(3.0))

{ lambda ; a:f32[]. let
    b:f32[] = log a
    c:f32[] = log 2.0
    d:f32[] = div b c
  in (d,) }


In [3]:
def log2_with_print(x):
  print("printed x:", x)
  ln_x = jnp.log(x)
  ln_2 = jnp.log(2.0)
  return ln_x / ln_2

print(jax.make_jaxpr(log2_with_print)(3.))

printed x: Traced<ShapedArray(float32[], weak_type=True)>with<DynamicJaxprTrace>
{ lambda ; a:f32[]. let
    b:f32[] = log a
    c:f32[] = log 2.0
    d:f32[] = div b c
  in (d,) }


In [4]:
print("hello")

hello


In [5]:

from jax import random, grad, jit

# Generate toy 2D data (two Gaussian blobs) -----
key = random.PRNGKey(0)
n_per_class = 250

key1, key2, key3 = random.split(key, 3)
mean0 = jnp.array([-1.0, -1.0])
mean1 = jnp.array([+1.0, +1.0])

X0 = mean0 + 0.6 * random.normal(key1, (n_per_class, 2))
X1 = mean1 + 0.6 * random.normal(key2, (n_per_class, 2))
X = jnp.concatenate([X0, X1], axis=0)                             # (N, 2)
y = jnp.concatenate([jnp.zeros(n_per_class), jnp.ones(n_per_class)])  # labels in {0,1}

# Shuffle
perm = random.permutation(key3, X.shape[0])
X, y = X[perm], y[perm]

# -- Model & loss -----
def logits(params, X):
    w, b = params
    return X @ w + b  # (N,)

# Binary cross-entropy via softplus for stability:
# BCE = mean( softplus(logit) - y*logit )
def loss_fn(params, X, y, l2=0.0):
    z = logits(params, X)
    data_loss = jnp.mean(jnp.logaddexp(0.0, z) - y * z)
    reg = 0.5 * l2 * (jnp.sum(params[0] ** 2) + params[1] ** 2)
    return data_loss + reg

# --- Gradients (jax.grad) -----
loss_grad = grad(loss_fn)

# ---- Gradient descent loop -----
def accuracy(params, X, y):
    z = logits(params, X)
    yhat = (z > 0).astype(y.dtype)
    return jnp.mean(yhat == y)

@jit
def step(params, X, y, lr, l2):
    g_w, g_b = loss_grad(params, X, y, l2)
    w, b = params
    w = w - lr * g_w
    b = b - lr * g_b
    return (w, b)

# Init
params = (jnp.zeros(2), 0.0)   # (w, b)
lr = 0.5
l2 = 1e-3
num_steps = 400

for t in range(1, num_steps + 1):
    params = step(params, X, y, lr, l2)
    if t % 50 == 0:
        current_loss = loss_fn(params, X, y, l2)
        acc = accuracy(params, X, y)
        print(f"step {t:3d} | loss {current_loss:.4f} | acc {acc*100:.1f}%")

w, b = params
print("\nFinal params:")
print("w:", w, " b:", float(b))
print("Final train accuracy:", float(accuracy(params, X, y)))


Failed to get TPU metadata (tpu-env) from instance metadata for variable CHIPS_PER_HOST_BOUNDS: DEADLINE_EXCEEDED: Timeout was reached
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/gcp_metadata_utils.cc:84
learning/45eac/tfrc/runtime/env_var_utils.cc:50

Failed to get TPU metadata (tpu-env) from instance metadata for variable HOST_BOUNDS: DEADLINE_EXCEEDED: Timeout was reached
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/gcp_metadata_utils.cc:84
learning/45eac/tfrc/runtime/env_var_utils.cc:50

Failed to get TPU metadata (tpu-env) from instance metadata for variable ALT: DEADLINE_EXCEEDED: Timeout was reached
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/gcp_metadata_utils.cc:84
learning/45eac/tfrc/runtime/env_var_utils.cc:50

Failed to get TPU metadata (tpu-env) from instance metadata for variable WRAP: DEADLINE_EXCEEDED: Timeout was reached
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/gcp_metadata_utils.cc:84
learning/45eac/tfrc/ru

step  50 | loss 0.0682 | acc 99.2%
step 100 | loss 0.0535 | acc 99.2%
step 150 | loss 0.0484 | acc 99.2%
step 200 | loss 0.0459 | acc 99.2%
step 250 | loss 0.0445 | acc 99.2%
step 300 | loss 0.0437 | acc 99.2%
step 350 | loss 0.0431 | acc 99.2%
step 400 | loss 0.0427 | acc 99.4%

Final params:
w: [3.182745 3.107227]  b: 0.5888577103614807
Final train accuracy: 0.9940000176429749
